# Car Fuel Emissions Dataset Dashboard

> This dashboard helps you explore the $CO_2$ emissions of different car models and their fuel consumption.

<b>[Data](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64) Updated as of June 15, 2023</b>

In [1]:
import ipywidgets as widgets
from ipywidgets import interactive_output, interact
from menu import init_widgets, style, setup_menu, select_table
from menu import init_widgets, style, setup_menu, select_table, clean_electric_range
from dashboard import (
    Seaborn_Barplot,
    Boxplot_ggplot,
    Seaborn_Scatter,
    Histogram_ggplot,
    Seaborn_Boxplot,
)
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
from itables import init_notebook_mode, show
import ipyvuetify as v

In [ ]:
%%capture

%load_ext sql

%sql duckdb:///../data/database/car_data.duckdb

%config SqlMagic.displaycon = False

In [ ]:
years = %sql select DISTINCT(model_year) from all_vehicles
years = [model_year[0] for model_year in years]

makes = %sql select DISTINCT(make_) from all_vehicles
makes = [m[0] for m in makes]

classes = %sql select DISTINCT(vehicleclass_) from all_vehicles
classes = [c[0] for c in classes]

co2 = %sql select DISTINCT(co2_rating) from all_vehicles where co2_rating is not null
co2 = [c[0] for c in co2]
# convert to int
co2 = [eval(c) for c in co2]

vehicle_type = %sql select DISTINCT(vehicle_type) from all_vehicles
vehicle_type = [v[0] for v in vehicle_type]

## Interactive Table

> Multiple values for Vehicle Class can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.

In [ ]:
# flake8: noqa
def select_table_electric(vehicle_type, year, vehicle_class, make, co2):
    """
    Select table based on vehicle type

    Parameters
    ----------
        vehicle_type : str
            Vehicle type (fuel-only, hybrid, or electric)
        year : int
            Model year
        vehicle_class : list
            Vehicle class (compact, midsize, etc.)
        make : str
            Car manufacturer
        co2 : int
            CO2 rating (higher rating suggests lower CO2 emissions)
    """
    print("Performing query")
    query = select_table(vehicle_type, year, vehicle_class, make, co2)

    # Use JupySQL magic %sql to execute the query
    result = %sql {{query}}

    # Convert the result to a Pandas DataFrame
    df = result.DataFrame()

    clear_output(wait=True)

    show(df, classes="display nowrap compact")


(
    widget_vehicle_type,
    widget_year,
    widget_make,
    widget_vehicle_class,
    widget_co2,
) = init_widgets(years, makes, classes, vehicle_type, style)

tab = setup_menu(
    widget_vehicle_type, widget_year, widget_vehicle_class, widget_make, widget_co2
)  # noqa E501

output = interactive_output(
    select_table_electric,  # noqa f821
    {
        "vehicle_type": widget_vehicle_type,
        "year": widget_year,
        "vehicle_class": widget_vehicle_class,
        "make": widget_make,
        "co2": widget_co2,
    },
)

display(tab, output)

This dashboard contains 5 plots, created using the `ggplot` API, `seaborn`, and `ipywidgets`:

1. Bar Plot of Car Manufacturing Trends
2. Boxplot of Fuel Consumption and $CO2$ Emission for All Vehicle Types 
3. Scatter Plot of Electric Vehicle Ranges and Charging Time by Car Size and Model Year 
4. Histogram of $CO_2$ Emissions by Vehicle and Fuel Type
5. $CO_2$ Emissions of Hybrid and Fuel-Only US Car Brands by Transmission Type

## Bar Plot of Car Manufacturing Trends

In [ ]:
%%capture

%%sql --save q_1_hybrid_electric --no-execute
SELECT DISTINCT model_year, vehicle_type, COUNT(id) AS num_vehicles
FROM all_vehicles
WHERE vehicle_type = 'hybrid' OR vehicle_type = 'electric'
GROUP BY model_year, vehicle_type
ORDER BY num_vehicles DESC;

In [ ]:
%%capture

%%sql --save q_1_fuel --no-execute
SELECT DISTINCT model_year, vehicle_type, COUNT(id) AS num_vehicles
FROM fuel
GROUP BY model_year, vehicle_type
ORDER BY model_year;

In [ ]:
%%capture

hybrid_electric_count = %sql SELECT * FROM q_1_hybrid_electric
fuel_count = %sql SELECT * FROM q_1_fuel

hybrid_electric_count = pd.DataFrame(hybrid_electric_count)
hybrid_electric_count = hybrid_electric_count.sort_values(by=["model_year"])
fuel_count = pd.DataFrame(fuel_count)
fuel_count = fuel_count.sort_values(by=["model_year"])

In [ ]:
barplot = Seaborn_Barplot(fuel_count, hybrid_electric_count)
interact(barplot.draw_bar_year_count, data=barplot.radio_button);

## Boxplot of Fuel Consumption and $CO2$ Emission for All Vehicle Types

In [ ]:
%%capture

%%sql --save boxplot_fuel_consum --no-execute
SELECT fuelconsumption_city_l_100km::FLOAT as fuelconsumption_city_l_100km, fuelconsumption_hwy_l_100km::FLOAT as fuelconsumption_hwy_l_100km, fuelconsumption_comb_l_100km::FLOAT as fuelconsumption_comb_l_100km, co2emissions_g_km::FLOAT as co2emissions_g_km
FROM all_vehicles

In [ ]:
boxplot = Boxplot_ggplot()
interact(boxplot.fuel_co2_boxplot, columns=boxplot.selection_button);

## Scatter Plot of Electric Vehicle Ranges and Charging Time by Car Size and Model Year 

In [ ]:
%%capture

%%sql --save electric_range_charge --no-execute
SELECT range1_km, recharge_time_h, vehicleclass_, model_year
FROM electric

In [ ]:
%%capture

electric_range = %sql SELECT * FROM electric_range_charge

electric_range = pd.DataFrame(electric_range)

clean_electric_range = clean_electric_range(electric_range)

In [ ]:
scatter = Seaborn_Scatter(clean_electric_range)
interact(scatter.draw_scatter_electric_range, hue=scatter.dropdown);

## Histogram of $CO_2$ Emissions by Vehicle and Fuel Type

In [ ]:
%%capture

%%sql --save hist_co2 --no-execute
SELECT vehicle_type, mapped_fuel_type, co2emissions_g_km::INTEGER as co2emissions_g_km
FROM all_vehicles
WHERE co2emissions_g_km is not null 

In [ ]:
histogram = Histogram_ggplot()
interact(
    histogram.co2_histogram,
    b=histogram.intslider,
    cmap=histogram.dropdown,
    fill=histogram.radio_button,
);

## $CO_2$ Emissions of Hybrid and Fuel-Only US Car Brands by Transmission Type

In [ ]:
%%capture

%%sql --save co2_usa --no-execute
SELECT vehicle_type, make_, co2emissions_g_km, transmission_type
FROM all_vehicles
WHERE co2emissions_g_km is not null AND
vehicle_type IN ('fuel-only', 'hybrid') AND
make_ IN ('cadillac', 'chevrolet', 'chrysler', 'ford', 'jeep', 'lincoln')

In [ ]:
%%capture

co2_usa = %sql SELECT * FROM co2_usa
co2_usa = pd.DataFrame(co2_usa)

# convert co2 to float

co2_usa["co2emissions_g_km"] = pd.to_numeric(co2_usa["co2emissions_g_km"])

In [ ]:
boxplot = Seaborn_Boxplot(co2_usa)
interact(boxplot.draw_boxplot_usa, hue=boxplot.dropdown);

## References

${^1}{^,}{^2}$ Canada, Service. “Government of Canada.” Service Canada, n.d. https://www.canada.ca/. 

$^3$ The 2020 EPA Automotive Trends Report: Greenhouse gas emissions, fuel ..., n.d. https://www.epa.gov/sites/default/files/2021-01/documents/420r21003.pdf. 